In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip /content/gdrive/MyDrive/archive.zip

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import callbacks
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image

In [ ]:
train_data = '/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_data = '/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid'

In [ ]:
# Define data generators with data augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


In [ ]:
train_gen = train_datagen.flow_from_directory(
    train_data,
    target_size=(256, 256),
    batch_size=64,
    class_mode='sparse'
)
test_gen = test_datagen.flow_from_directory(
    valid_data,
    target_size=(256, 256),
    batch_size=64,
    class_mode='sparse'
)

In [ ]:
# definisikan model transfer learning dan regulasi
base_model = keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(256, 256, 3)
)

for layer in base_model.layers[:-4]:
    layer.trainable = False

model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dropout(0.5), #mengurangi overfitting
    keras.layers.Dense(38, activation='softmax')
])

opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3),
    tf.keras.callbacks.ModelCheckpoint(filepath='/content/gdrive/MyDrive/Model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

history = model.fit(train_gen, validation_data=test_gen, epochs=25, callbacks=my_callbacks)

In [ ]:
plt.figure(figsize= (20,5))
cm = confusion_matrix(labels, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=list(range(1,39)))
fig, ax = plt.subplots(figsize=(15,15))
disp.plot(ax=ax,colorbar= False,cmap = 'YlGnBu')
plt.title("Confusion Matrix")
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.savefig('confusion_matrix.png')
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load trained model
model_path = '/content/gdrive/MyDrive/Model.18-0.08.h5'  # Ganti dengan path model yang ingin Anda gunakan
model = load_model(model_path)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
model_path = '/content/gdrive/MyDrive/Model/model.tflite'
with open(model_path, 'wb') as f:
    f.write(tflite_model)

In [ ]:
# Input gambar untuk prediksi
image_path = '/content/OIP (3).jpg'  # Ganti dengan path gambar yang ingin Anda uji
image = load_img(image_path, target_size=(256, 256))
image_array = img_to_array(image)
image_array = image_array / 255.0
image_array = np.expand_dims(image_array, axis=0)

# Melakukan prediksi dengan model
predictions = model.predict(image_array)
class_index = np.argmax(predictions[0])

print("Prediksi kelas:", class_index)

1/1 [==============================] - 0s 61ms/step
Prediksi kelas: 0


In [ ]:
# Input gambar untuk prediksi
image_path = '/content/OIP (1).jpg'  # Ganti dengan path gambar yang ingin Anda uji
image = load_img(image_path, target_size=(256, 256))
image_array = img_to_array(image)
image_array = image_array / 255.0
image_array = np.expand_dims(image_array, axis=0)

# Melakukan prediksi dengan model
predictions = model.predict(image_array)
class_index = np.argmax(predictions[0])

print("Prediksi kelas:", class_index)

1/1 [==============================] - 0s 27ms/step
Prediksi kelas: 20


In [ ]:
# Input gambar untuk prediksi
image_path = '/content/grape-leaf-white-background-grape-leaf-white-background-118078027.webp'  # Ganti dengan path gambar yang ingin Anda uji
image = load_img(image_path, target_size=(256, 256))
image_array = img_to_array(image)
image_array = image_array / 255.0
image_array = np.expand_dims(image_array, axis=0)

# Melakukan prediksi dengan model
predictions = model.predict(image_array)
class_index = np.argmax(predictions[0])

print("Prediksi kelas:", class_index)

1/1 [==============================] - 0s 26ms/step
Prediksi kelas: 14
